In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent  # 找出根目錄：Path.cwd()找出現在所在目錄(/run).parent(上一層是notebook).parent(再上層一層business_district_discovery)
print(project_root)
sys.path.append(str(project_root))

In [ ]:
from utils.addr_parser import AddressParser
from utils.addr_splitter import AddressSplitter

from utils.hepler_poi import corrections, company_name
from utils.hepler_poi import csv_extractor, convert_columns, apply_manual_corrections, process_dataframe_addresses
from utils.hepler_poi import enrich_chain_status, split_chain_company_name, fill_company_and_branch_by_id

In [ ]:
# 全國營業(稅籍)登記資料集原始資料
csv_path = r"C:\labs\geo-grid\data\raw\National_Business_Registration_Dataset_202510.csv"
extracted =csv_extractor(csv_path)
print(f" 逐筆交易資料載入成功: {extracted.shape}  記憶體使用: {extracted.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
df = extracted.loc[extracted['統一編號'].notna(),
    ['營業地址', '統一編號', '總機構統一編號', '營業人名稱', '資本額', '設立日期', '組織別名稱', '使用統一發票', '行業代號', '名稱']
].copy()

In [ ]:
df = convert_columns(df)

In [ ]:
splitter = AddressSplitter(
    input_column='營業地址',
    output_column='正規化營業地址',
    status_column='地址狀態',
    delimiters=None,             # 使用預設分隔符
    batch_size=10000,
    print_report=True,
    preprocessor=None          # 預設不做任何前處理
)

df_proc, changes = splitter.process(df)

In [ ]:
parser = AddressParser()
df_proc = parser.process(df_proc)

In [ ]:
# 資料補正
df_proc = apply_manual_corrections(
    df=df_proc,
    corrections=corrections
)

In [ ]:
## 暫時不知用途為何？
# df_proc = process_dataframe_addresses(
#     df_proc,                    # 你的DataFrame
#     address_column='正規化營業地址',  # 原始地址欄位名稱
#     normalized_column='清理地址',    # 新增的正規化地址欄位名稱
#     floor_column='樓層資訊'         # 新增的樓層欄位名稱
# )

In [ ]:
# 篩選餐飲業poi
catering_services = df_proc.loc[
    # 1. 篩選特定縣市
    (df_proc['縣市'].isin(['臺北市', '新北市', '桃園市', '臺中市', '臺南市', '高雄市', '新竹市', '新竹縣'])) & 
    # 2. 篩選中類為餐飲業
    (df_proc['中類'] == '餐飲業') & 
    # 3. 排除特定名稱 (注意波浪符號的位置)
    (~df_proc['名稱'].isin(['外燴（辦桌）承包', '其他外燴及團膳承包', '團膳承包', '學校營養午餐供應']))
]

In [ ]:
# 連鎖/非連鎖
catering_services = enrich_chain_status(catering_services)

In [ ]:
# 折分公司名稱
catering_services = split_chain_company_name(
    catering_services,
    keyword=["股份有限公司", "有限公司"]
)

In [ ]:
# 公司名稱/分公司名稱補正
catering_services = fill_company_and_branch_by_id(
    catering_services,
    mapping=company_name,
    id_col="統一編號",
    company_col="公司名稱",
    branch_col="分公司名稱",
)

In [ ]:
catering_services.loc[
    (catering_services['公司名稱'].isna())&
    (catering_services['營業型態'] == '連鎖'),
].sort_values(by='總機構統一編號')

In [ ]:
# catering_services[['名稱']].drop_duplicates()